## Basic prompting

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain_ollama import ChatOllama

model = ChatOllama(model="gpt-oss:20b", temperature=1)

agent = create_agent(model=model)

question = HumanMessage(content="What's the capital of the moon?")

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

There isn’t one—no country, no government, no permanent settlement, so the Moon has no capital.  
The only “landmarks” are historic sites like Apollo 11’s Tranquility Base, but those are scientific landing sites, not a capital city. If humanity ever builds a lunar colony, it will decide its own capital—until then, the Moon stays a land without a capital.


In [3]:
system_prompt = "You are a science fiction writer, create a capital city at the users request."

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

**Luna Prime – The Orbital Capital of the Lunar Federation**

When the first basaltic outpost on the Moon’s near side, *Artemis‑I*, burst into civilization in 2087, the idea of a “capital” felt as alien to the colony as the star‑light that bathed its regolith. By the time the Lunar Federation was formally established in 2135, the idea had evolved from a mere hub of administrative activity into a city‑state that redefined what it means to be a metropolis in space.

---

### 1. Location: The **Rima C.** Basin

Luna Prime sits at the western edge of the **Rima C.** rille, a wide, ancient fault line that stretches 120 km across the Moon’s surface. The basin offers a unique combination of advantages:

- **Low Gravity & Regolith Stability:** The basaltic crust is surprisingly compact, allowing the construction of vertical towers that would tip on Earth’s surface.
- **Solar Access:** The basin’s gentle slope channels morning and evening sunlight into a shallow “solar canyon,” enabling a conti

## Few-shot examples

In [4]:
system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

User: What is the capital of mars?
Scifi Writer: Marsialis

User: What is the capital of Venus?
Scifi Writer: Venusovia

"""

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

Lunavia


## Structured prompts

In [5]:
system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

Please keep to the below structure.

Name: The name of the capital city

Location: Where it is based

Vibe: 2-3 words to describe its vibe

Economy: Main industries

"""

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

**Name:** Selene Prime  
**Location:** The southern polar region, nestled within Shackleton Crater, with extensive ice deposits and a network of orbital tethered habitats.  
**Vibe:** Quiet, resilient, forward‑thinking.  
**Economy:** Lunar resource extraction (helium‑3, regolith mining), high‑tech research facilities (astronomy, vacuum physics), and orbital logistics hub for interplanetary trade.


## Structured output

In [10]:
from pydantic import BaseModel
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

class CapitalInfo(BaseModel):
    name: str
    location: str
    vibe: str
    economy: str

# Wrap model with structured output
model = ChatOllama(model="gpt-oss:20b", temperature=1).with_structured_output(CapitalInfo)

# Create a simple prompt + model chain
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a science fiction writer, create a capital city at the user's request."),
    ("human", "{input}")
])

chain = prompt | model

# Invoke
response = chain.invoke({"input": "What is the capital of The Moon?"})
print(response)  # ✅ CapitalInfo object

name='Luna Nova' location='South‑Pole sector of the Mare Tranquillitatis, nestled within the towering rim of the Shackleton‑Curry crater' vibe='A gleaming orbital‑hull sprawl, its streets lit by soft bioluminescent panels and the ever‑present glow of Earth above' economy='Hub for regolith mining, hydroponic research, and quantum‑link communications with Mars and Earth, powered by a nuclear fusion array buried beneath the regolith\n\n---\n\n**Luna Nova** is the beating heart of lunar civilization.  Conceived in the early 23rd century as the first fully self‑sustaining orbital habitat, it grew into the capital of the Moon by the late 25th century.  Its twin towers—*The Helix* and *The Spiral*—rise like twin guardians over the icy southern plains, their surfaces covered in a reflective alloy that refracts the sun’s rays into a spectrum of soft blues and greens.  The city’s infrastructure is a maze of sky‑bridges and modular habitat modules that can be reconfigured as the needs of the luna

In [11]:
print(response.name)  # Access fields directly

Luna Nova


In [ ]:
from langchain.agents import create_tool_calling_agent
from langchain_core.tools import tool

@tool
def dummy_tool(query: str) -> str:
    """A dummy tool for demonstration."""
    return f"Tool result for: {query}"

# Model with structured output
model = ChatOllama(model="llama3.1").with_structured_output(CapitalInfo)

# Create tool-calling agent (note: structured output + tools is complex; test carefully)
agent = create_tool_calling_agent(
    model=model,
    tools=[dummy_tool],
    prompt="You are a science fiction writer, create a capital city at the user's request."
)

question = HumanMessage(content="What is the capital of The Moon?")

response = agent.invoke(
    {"messages": [question]}
)

response["structured_response"]

In [7]:
from pydantic import BaseModel

class CapitalInfo(BaseModel):
    name: str
    location: str
    vibe: str
    economy: str

agent = create_agent(
    model=model,
    system_prompt="You are a science fiction writer, create a capital city at the users request.",
    response_format=CapitalInfo
)

question = HumanMessage(content="What is the capital of The Moon?")

response = agent.invoke(
    {"messages": [question]}
)

response["structured_response"]

TypeError: Client.chat() got an unexpected keyword argument 'response_format'

In [7]:
response["structured_response"].name

'The Moon'

In [8]:
capital_info = response["structured_response"]

capital_name = capital_info.name
capital_location = capital_info.location

print(f"{capital_name} is a city located at {capital_location}")

The Moon is a city located at Lunar Surface
